### There are two related tab-delimited vcf files. 
-regular
-annotated

### This notebook cleans up the annotated dataframe and saves it as pickled dataframe

In [34]:
import csv
import gzip
import re
import pickle

import pandas as pd

In [2]:
# with gzip.open("vep/clinvar.annotated.vcf.gz", "rt") as f:
#         for line in f:
#             if line.startswith("##INFO=<ID=CSQ"):
#                 m = re.search(r'.*Format: (.*)">', line)
#                 cols = m.group(1).split("|")
#                 continue

#             if line.startswith("#"):
#                 continue
#             record = line.split("\t")
#             (
#                 chromosome,
#                 position,
#                 clinvar_id,
#                 reference_base,
#                 alternate_base,
#                 qual,
#                 filter_,
#                 info,
#             ) = record
#             info_field = info.strip("\n").split(";")

#             # to lookup in clivnar_annotaitons
#             clinvar_id = int(clinvar_id)

#             # only keep the variants that have been evaluated by multiple submitters
#             if clinvar_id in clinvar_annotations:
#                 # initialize a dictionary to hold all the VEP annotation data
#                 annotation_data = {column: None for column in cols}
#                 annotation_data.update(clinvar_annotations[clinvar_id])
#                 # fields directly from the vcf
#                 annotation_data["CHROM"] = str(chromosome)
#                 annotation_data["POS"] = position
#                 annotation_data["REF"] = reference_base
#                 annotation_data["ALT"] = alternate_base

#                 for annotations in info_field:
#                     column, value = annotations.split("=")

#                     if column == "CSQ":
#                         for csq_column, csq_value in zip(cols, value.split("|")):
#                             annotation_data[csq_column] = csq_value
#                         continue

#                     annotation_data[column] = value
#                 dw.writerow(annotation_data)

In [3]:
# get columns names from tab delimited file
cv_columns = {}
with gzip.open("raw_vcf/clinvar.annotated.vcf.gz", "rt") as f:
    for metaline in f:
        if metaline.startswith("##INFO"):
            colname = re.search("ID=(\w+),", metaline.strip("#\n"))
            coldesc = re.search(".*Description=(.*)>", metaline.strip("#\n"))
            cv_columns[colname.group(1)] = coldesc.group(1).strip('"')

In [4]:
# read tab delimited
cv_df = pd.read_csv(
    "raw_vcf/clinvar.annotated.vcf.gz",
    sep="\t",
    skiprows=35,
    usecols=[0, 1, 2, 3, 4, 7], # rid of columns 5, 6
    header=None,
)

In [5]:
cv_df.rename(columns={0: "CHROM", 1: "POS", 2: "ID", 3: "REF", 4: "ALT"}, inplace=True)

In [6]:
# convert the long dictionary in column 7 to actual columns
def list_to_dict(l):
    """Convert list to dict."""
    return {k: v for k, v in (x.split("=") for x in l)}

cv_df = pd.concat(
    [
        cv_df.drop([7], axis=1),
        cv_df[7].str.split(";").apply(list_to_dict).apply(pd.Series),
    ],
    axis=1,
)

In [25]:
# the CSQ column has a bit different syntax. Unpack and convert into actual columns

# get column names for CSQ
with gzip.open("raw_vcf/clinvar.annotated.vcf.gz", "rt") as f:
        for line in f:
            if line.startswith("##INFO=<ID=CSQ"):
                m = re.search(r'.*Format: (.*)">', line)
                cols = m.group(1).split("|")

# pipe to dict
def CSQ_to_dict(l):
    '''
    Convert the pipe_separated values in the CSQ column to dict
    '''
    annotation_data = {}
    for csq_column, csq_value in zip(cols, l):
        annotation_data[csq_column] = csq_value
    return annotation_data
        
# convert and concat
cv_df = pd.concat(
    [
        cv_df.drop(['CSQ'], axis=1),
        cv_df['CSQ'].str.split("|").apply(CSQ_to_dict).apply(pd.Series),
    ],
    axis=1,
)

In [36]:
file = open('cv_df_extracted', 'wb')
pickle.dump(cv_df, file)
file.close()